In [52]:
import numpy as np

window_size = 11
from kl.utils import load_fx
X, y, returns = load_fx(data_start=0, data_end=5000, window_size=window_size, shift=1)

In [53]:
import torch
# Check for available device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA for GPU acceleration")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon) for GPU acceleration")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using MPS (Apple Silicon) for GPU acceleration


In [54]:
import torch
import torch.nn as nn
import torch.optim as optim

# SVM-like classifier (linear model)
class SVMClassifier(nn.Module):
    def __init__(self, input_dim):
        super(SVMClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, 1)  # Linear classifier with one output for binary classification

    def forward(self, x):
        return self.fc(x)  # No activation, similar to SVM's linear decision boundary

# Hinge loss function (for SVM-like behavior)
def hinge_loss_fn(y_pred, y_true):
    return torch.mean(torch.clamp(1 - y_pred * y_true, min=0))

In [55]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.svm import SVC
from sklearn.metrics import hinge_loss
from kl.utils import load_fx
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def reparameterize(mu, log_var):
    std = torch.exp(0.5 * log_var)  # Compute standard deviation from log variance
    eps = torch.randn_like(std)     # Sample from standard normal distribution
    return mu + eps * std  # Reparameterization: z = mu + sigma * epsilon


# Define the quantum circuit for 8 qubits (one per feature)
def quantum_circuit(params, x):
    n_qubits = len(x)  # Ensure we are only working with 8 qubits
    for i in range(n_qubits):
        qml.RX(x[i], wires=i)  # Apply RX to qubit i with the i-th feature
        qml.RY(params[0], wires=i)
        qml.RZ(params[1], wires=i)
    
    # Measure the expectation value of Pauli-Z on all qubits
    # Return only a single vector of length 8 (one value per qubit)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class QuantumEncoder(nn.Module):
    def __init__(self, n_qubits, n_features):
        super(QuantumEncoder, self).__init__()
        self.n_qubits = n_qubits
        self.n_features = n_features
        
        # Quantum circuit and device
        self.dev = qml.device("default.qubit", wires=n_qubits)
        self.qcircuit = qml.QNode(quantum_circuit, self.dev)

        # Additional layers to output mean and log variance for latent space
        self.fc_mu = nn.Linear(n_qubits, n_qubits)      # Mean for latent space
        self.fc_logvar = nn.Linear(n_qubits, n_qubits)  # Log variance for latent space

    def forward(self, x):
        outputs = []
        params = torch.randn(2)  # Parameters for the quantum circuit
        
        # Process each sample in the batch
        for sample in x:
            sample = sample.detach().cpu().numpy()
            output = self.qcircuit(params, sample)
            outputs.append(np.array(output))
        
        outputs_np = np.stack(outputs, axis=0)
        
        # Move latent_vector to the device where the model is located
        latent_vector = torch.tensor(outputs_np, dtype=torch.float32).to(x.device)
        
        # Generate mean and log variance
        mu = self.fc_mu(latent_vector)
        log_var = self.fc_logvar(latent_vector)

        return mu, log_var  # Output mean and log variance for reparameterization
    
# Define the classical decoder
class ClassicalDecoder(nn.Module):
    def __init__(self, n_qubits, n_features):
        super(ClassicalDecoder, self).__init__()
        self.fc1 = nn.Linear(n_qubits, 128)  # Change input size from 1 to 8 to match latent vector size
        self.fc2 = nn.Linear(128, n_features)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Decode the quantum output
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [56]:
class QuantumVAE(nn.Module):
    def __init__(self, n_qubits, n_features):
        super(QuantumVAE, self).__init__()
        self.encoder = QuantumEncoder(n_qubits, n_features)  # Quantum encoder
        self.decoder = ClassicalDecoder(n_qubits, n_features)  # Classical decoder
        self.classifier = SVMClassifier(n_qubits)  # Embedded SVM classifier

    def forward(self, x):
        # Get mean and log variance from the encoder
        mu, log_var = self.encoder(x)

        # Reparameterization trick to sample latent vector z
        latent = reparameterize(mu, log_var)

        # Get reconstructed data and SVM classification output
        reconstructed_x = self.decoder(latent)
        classification_output = self.classifier(latent)

        # Return latent vector, reconstructed data, classification output, and parameters for KL divergence
        return latent, reconstructed_x, classification_output, mu, log_var

In [57]:
def compute_class_variance(latent_vectors, labels):
    # Calculate variance of latent vectors for each class
    unique_labels = torch.unique(labels)
    class_variances = {}
    
    for label in unique_labels:
        class_latents = latent_vectors[labels == label]
        class_variance = torch.var(class_latents, dim=0).mean()  # Average variance across all latent dimensions
        class_variances[label.item()] = class_variance
    
    return class_variances

In [58]:
def train_vae_with_svm(vae, dataloader, optimizer, n_epochs=10):
    vae.train()

    for epoch in range(n_epochs):
        total_loss = 0
        for batch in dataloader:
            # Unpack batch data (features and labels)
            x, labels = batch
            x, labels = x.to(device), labels.to(device)  # Move data to the device

            # Forward pass through VAE
            latent_vectors, reconstructed_x, svm_predictions, mu, log_var = vae(x)

            # Compute variance for each class in the latent space
            class_variances = compute_class_variance(latent_vectors, labels)

            # Reconstruction loss
            recon_loss = nn.MSELoss()(reconstructed_x, x)

            # KL divergence
            kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

            # Initialize total SVM loss
            svm_loss = 0
            for label in class_variances:
                variance_weight = class_variances[label]
                class_indices = (labels == label).unsqueeze(1).bool()  # Ensure class_indices is boolean
                svm_loss += variance_weight * hinge_loss_fn(svm_predictions[class_indices], class_indices.float())

            # Total loss (reconstruction + weighted KL divergence + SVM loss)
            total_batch_loss = recon_loss + kl_divergence + 0.1 * svm_loss

            # Backpropagation and optimization
            optimizer.zero_grad()
            total_batch_loss.backward()
            optimizer.step()

            total_loss += total_batch_loss.item()

        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss/len(dataloader):.4f}, KL: {kl_divergence.item():.4f}, SVM Loss: {svm_loss:.4f}')

In [ ]:
# Initialize the Quantum VAE
vae_model = QuantumVAE(n_qubits=X.shape[1], n_features=X.shape[1])  # Adjust the number of qubits and features based on your data

# Move the VAE model to the appropriate device
vae_model = vae_model.to(device)

# Define the optimizer
optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)

# Example DataLoader (assuming you have X and y)
dataloader = DataLoader(TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)), batch_size=64, shuffle=True)

# Train the VAE with the embedded SVM classifier
train_vae_with_svm(vae_model, dataloader, optimizer, n_epochs=100)

Epoch [1/100], Loss: 47.6658, KL: 30.4958, SVM Loss: 2.2457
Epoch [2/100], Loss: 19.3705, KL: 15.7063, SVM Loss: 2.0265


In [ ]:
def inference(vae, x):
    vae.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient computation for inference
        x = x.to(device)  # Move the input data to the device
        latent_vectors, reconstructed_x, svm_predictions, mu, log_var = vae(x)
        
        class_labels = (svm_predictions > 0).float()  # Assuming binary classification
        return reconstructed_x, class_labels, latent_vectors, mu, log_var

In [ ]:
X_test = X
# Example input data for inference (can be a batch of new data)
new_data = torch.tensor(X_test, dtype=torch.float32)

# Perform inference
reconstructed_data, predicted_labels, latent_vectors, mu, log_var = inference(vae_model, new_data)

In [ ]:
predicted_labels_np = predicted_labels.cpu().detach().numpy()
reconstructed_data_np = reconstructed_data.cpu().detach().numpy()
latent_vectors_np = latent_vectors.cpu().detach().numpy()

In [ ]:
np.savetxt('reconstructed_data.txt', reconstructed_data_np)
# Print results
# print("Reconstructed Data:\n", reconstructed_data)
# print("Predicted Labels:\n", predicted_labels)
# print("Latent Vectors:\n", latent_vectors)